**CPLN 680 Capstone Project**

Rui Jiang

This notebook aims to save scripts I used to do the analysis and data visualization

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

#read file
file = r'C:\Users\jrach\Desktop\Jiang_Rui_RecreationPhilly\raw_data\NE_CSO.xlsx'###
data = pd.read_excel(file)
print("raw:",data)
#convert date time to date(year,month,day)
data["Date"] = data["Datetime [EST\EDT]"].dt.date

# use date to sum up daily volume for each CSO outfall
data = data.groupby(['Date']).sum()

#check
print("check:",data)

#write the output excel file
name = "CSOdaily2019.xlsx"###
savefile=data.to_excel(name)
print("Excel file created sucessfully")

In [ ]:

#plt.plot(x,y);
#plt.plot(x, y, 'r--o')
#plt.xlabel('x')
#plt.ylabel('y')
#plt.title('title');
#plt.plot(x, y, 'r--o', x, y ** 1.1, 'bs', x, y ** 1.2, 'g^-' );
print(data)

In [ ]:
plt.plot(data.iloc[:,1])

In [ ]:
import pandas as pd
import numpy as np
import statistics as st

#comments
###changable


def universal_excelsheet(file, columnNameforDate, columnNameforBacteria, columnNameforLocation, columnforData):
    exceldata = pd.read_excel(file)
    exceldf = pd.DataFrame(exceldata)
    exceldf.rename(columns={columnNameforDate: 'Date', columnNameforBacteria: 'Bacteria',
                            columnNameforLocation: 'Location', columnforData: 'Data'}, inplace=True)
    exceldf = exceldf.dropna(subset=['Date', 'Bacteria', 'Location', 'Data'])
    exceldf.to_excel('data.xlsx')
    return exceldf

### FIT EXCEL SHEET
#DRBC 2019
#excel = '/Nearshore_Bacteria.xlsx'
#PWD
#excel = '/Howard_Neukrug_Water_Center_PWD_Shore_Grab_Bacteria_2019.xlsx'
#DRBC 2020
excel = 'DRBC 2020 nearshore.xlsx'


### FIT THE COLUMNS NAMES OF ABOVE EXCEL SHEET
#DRBC
#ExcelSheet(excel,'ActivityStartDate','CharacteristicName','Location','ResultMeasureValue')
#PWD
#ExcelSheet(excel,'sample.date', 'parameter', 'loc.ID', 'data.value')
#DRBC 2020
universal_excelsheet(excel, 'CollectionDate', 'Parameter', 'SiteName', 'result')

#Excel Sheet made from function
data = pd.read_excel(r'/data.xlsx')

#### Create a pandas dataframe
df = pd.DataFrame(data)

# Geometric Mean
def geo_mean(iterable):
    a = np.array(iterable)
    return a.prod()**(1.0/len(a))

# Statistical Threshold Value
def stv(iterable):
    # log based 10 : STV = 10 ** ( avg(log values) + 1.282 * std(log values) )
    b = np.log10(iterable)
    eq = 10 ** (st.mean(b) + (1.282 * st.stdev(b)))
    return eq

# Average
def average(iterable):
    a = np.array(iterable)
    eq = (a.sum())/(len(iterable))
    return eq

# Create a list of lists for the bacteria, location, month, etc
LIST = []

# month, bacteria, and location -> geometric mean, STV, average, and percentiles
# exports into a new pandas dataframe
def pullingdata(month,year,bacteria,locationID):
    # Matching the correct columns with the parameters from the function
    DATA = (df.loc[(df['Date'].dt.month == (month))
                   & (df['Date'].dt.year == (year))
                   & (df['Bacteria'] == (bacteria))
                   & (df['Location'] == (locationID))])

    # Make a list of all of the data (MPN/100mL) with the same criteria (month, bacteria, and location iD
    DataList = DATA['Data'].tolist()
    DayList = DATA['Date'].dt.day.tolist()

    # If there is no data for the month, only print this statement and do nothing else
    if not DataList:
        print("No data for month " + str(month) + "\n")

    elif len(DataList) == 1:
        print("Bacteria: " + str(bacteria))
        print("Location ID: " + str(locationID))
        print("Month/Year: " + str(month) + "/" + str(year))
        print(DataList)
        print(DayList)
        print(average(DataList))
        print("Only one data point - cannot calculate STV, GM, or percentiles\n")
        x = 'NA'
        return LIST.append([month, year, locationID, float(average(DataList)), bacteria,x,x,x,x,x,DataList[0],x,x,x,DataList,DayList])

    # If there is data for the month, make a list of the month, location, bacteria, and the geometric mean
    else:
        print("Bacteria: " + str(bacteria))
        print("Location ID: " + str(locationID))
        print("Month/Year: " + str(month) + "/" + str(year))
        print(DataList)
        print(DayList)
        print(average(DataList))
        print("Geometric Mean: " + str(geo_mean(DataList)))
        print("STV: " + str(stv(DataList)) + "\n")
        p0 = np.percentile(DataList, 0)
        p5 = np.percentile(DataList, 5)
        p25 = np.percentile(DataList, 25)
        p50 = np.percentile(DataList, 50)
        p75 = np.percentile(DataList, 75)
        p90 = np.percentile(DataList, 90)
        p100 = np.percentile(DataList, 100)
        return LIST.append([month, year, locationID, bacteria, float(average(DataList)), float(geo_mean(DataList)), float(stv(DataList)),
                            float(p0),float(p5),float(p25),float(p50),float(p75),float(p90),float(p100),
                            DataList,DayList])

# Make a list of only the unique locations and bacteria types
LocationList = df['Location'].unique().tolist()
BacteriaList = df['Bacteria'].unique().tolist()
YearList = df['Date'].dt.year.unique().tolist()

i=0
#loop through all the years
for Y in range(len(YearList)):
    # outer loop is for all the months
    while i < 13:
        # middle loop is running through all the unique locations
        for L in range(len(LocationList)):
            # inner loop is running through all of the unique bacteria
            for B in range(len(BacteriaList)):
                # running the above function
                pullingdata(i,YearList[Y],BacteriaList[B],LocationList[L])
        i+=1



exportingdata = pd.DataFrame(columns = ["MONTH", "YEAR", "LocationID","Bacteria","Average","GEO MEAN","STV",
                                   "P0","P5","P25","P50","P75","P90","P100",
                                   "Data List","Day List"])


# make the earlier defined list into a dataframe
exportingdata = exportingdata.append(pd.DataFrame(LIST, columns = exportingdata.columns))


##### EXPORT TO EXCEL

### CHANGE THIS TO NAME
#file_name = 'PWDdata.xlsx'
#file_name = 'DRBC2020data.xlsx'

# saving the excel
exportingdata.to_excel(file_name)
print('Export to excel successfully.')


Rain Data

In [ ]:
import pandas as pd




# To make the important column names universal for all the bacterial data
def universal_excelsheet(file, ColumnNameforDate, ColumnNameforBacteria, ColumnNameforLocation, ColumnforData):
    data = pd.read_excel(file)
    df = pd.DataFrame(data)
    df.rename(columns={ColumnNameforDate: 'Date', ColumnNameforBacteria: 'Bacteria',
                       ColumnNameforLocation: 'Location', ColumnforData: 'Data'}, inplace=True)
    df = df.dropna(subset=['Date', 'Bacteria', 'Location', 'Data'])
    df.to_excel('data.xlsx')
    return df



excel = 'raw_data/FIB and Field Data1.xlsx'

universal_excelsheet(excel, 'Sample Date', 'Bacteria', 'Station', 'Result') 


# Excel Sheet made from function
data = pd.read_excel(r'/data.xlsx')
df = pd.DataFrame(data) # turn into pandas dataframe



def RainExcelSheet(file1999,file2005,file2008,file2019, file2020, file2021): # creates a separate excel sheet of all the rainfall information
    # read in two columns date and prcp from each year rainfall data file
    data2005 = pd.read_excel(file2005, usecols=["DATE", "PRCP"])
    data1999 = pd.read_excel(file1999, usecols=["DATE", "PRCP"]) 
    data2008 = pd.read_excel(file2008, usecols=["DATE", "PRCP"])
    data2019 = pd.read_excel(file2019, usecols=["DATE", "PRCP"])
    data2020 = pd.read_excel(file2020, usecols=["DATE", "PRCP"])
    data2021 = pd.read_excel(file2021, usecols=["DATE", "PRCP"])
    # turn into a pandas dataframe
    df1999 = pd.DataFrame(data1999) 
    df2005 = pd.DataFrame(data2005)
    df2008 = pd.DataFrame(data2008)
    df2019 = pd.DataFrame(data2019)
    df2020 = pd.DataFrame(data2020)
    df2021 = pd.DataFrame(data2021)
    # combine all the data frames
    frames = [df1999, df2005, df2008, df2019, df2020, df2021]
    df = pd.concat(frames)
    # do the rolling sum of the rainfall
    df['SumTwoDays'] = round(df['PRCP'].rolling(window=2).sum(), 4) 
    df['SumThreeDays'] = round(df['PRCP'].rolling(window=3).sum(), 4)
    df['SumSixDays'] = round(df['PRCP'].rolling(window=6).sum(), 4)
    df['SumTenDays'] = round(df['PRCP'].rolling(window=10).sum(), 4)
    # count the number of days after a certain rainfall event amount
    df['Count of days after .1 Rainfall Event'] = df.groupby((df['PRCP'] >= .1).cumsum()).cumcount()
    df['Count of days after any rainfall event'] = df.groupby((df['PRCP'] > 0).cumsum()).cumcount()
    # put all this into one excel sheet
    df.to_excel('RainData.xlsx')
    return df


# Excel sheet for the precipiation

excel1999 = r"RawData/rain1999to2004.xlsx"
excel2005 = r"RawData/rain2005to2007.xlsx"
excel2008 = r"RawData/rain2008to2018.xlsx"
excel2019 = r"raw_data/Philadelphia Airport Precip data 2019.xlsx"
excel2020 = r"RawData/RainData2020.xlsx"
excel2021 = r"RawData/NOAARainfall2021.xlsx"

# combine all the rainfall datasheets I have into one complete excel file in using the RainExcelSheet function
RainExcelSheet(excel1999, excel2005, excel2008, excel2019, excel2020, excel2021)
raindata = pd.read_excel(r'RainData.xlsx')
df1 = pd.DataFrame(raindata)

# Function that gets a dataframe and turns it into a dictionary
def dataframe_to_dict(df, key_column, value_column):
    dict = df.set_index(key_column)[value_column].to_dict()
    return dict

# turns this information into a dictionary in order to add it into another dataset
rain_dict = dataframe_to_dict(df1, 'DATE', 'PRCP')
TwoDays_dict = dataframe_to_dict(df1, 'DATE', 'SumTwoDays')
ThreeDays_dict = dataframe_to_dict(df1, 'DATE', 'SumThreeDays')
SixDays_dict = dataframe_to_dict(df1, 'DATE', 'SumSixDays')
TenDays_dict = dataframe_to_dict(df1, 'DATE', 'SumTenDays')
Count_dict = dataframe_to_dict(df1, 'DATE', 'Count of days after .1 Rainfall Event')
Count_dict2 = dataframe_to_dict(df1, 'DATE', 'Count of days after any rainfall event')

DAILYLIST = (df['Date']) # makes a list of all the dates

# make a list of lists for all of the rain values
totalrain = []
totaltwo = []
totalthree = []
totalsix = []
totalten = []
totalcount = []
totalcount2 = []
for DateInList in DAILYLIST:
    print(DateInList)
    rain = rain_dict.get(DateInList)
    two = TwoDays_dict.get(DateInList)
    print(two)
    three = ThreeDays_dict.get(DateInList)
    print(three)
    six = SixDays_dict.get(DateInList)
    print(six)
    ten = TenDays_dict.get(DateInList)
    print(ten)
    count = Count_dict.get(DateInList)
    print(count)
    count2 = Count_dict2.get(DateInList)
    print(count2)
    totalrain.append(rain)
    totaltwo.append(two)
    totalthree.append(three)
    totalsix.append(six)
    totalten.append(ten)
    totalcount.append(count)
    totalcount2.append(count2)

# This makes the excel sheet into separate columns with the following titles, so each rainfall amount is its own column    
exportingrain = (pd.DataFrame({'Rain per Each Day': totalrain, 'Sum of Rain for Two Days': totaltwo,
                               'Sum of Rain for Three Days': totalthree, 'Sum of Rain for Six Days': totalsix,
                               'Sum of Rain for Ten Days': totalten,
                               'Count of days after .1 Rainfall Event': totalcount,
                               'Count of days after any rainfall event': totalcount2}))

# join the two data
exportingdata = pd.concat([df, exportingrain], axis=1)

##### EXPORT TO EXCEL


file_name = 'Summer2021Data_Rainfall.xlsx'

# saving the excel
exportingdata.to_excel(file_name)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats


data = pd.read_excel(r'/Water Research/DRBC 2019,2020,2021/DRBC 19,20,21.xlsx')
#print(data)

df = pd.DataFrame(data)

Data = df.loc[(df['Bacteria'] == ('E.Coli'))]

DataList = Data['Data'].tolist()

LogData = np.log10(DataList).tolist()

#plt.hist(LogData, bins=25)
#plt.show()

# list of distributions in scipy
list_of_dists = ['alpha', 'anglit', 'arcsine', 'beta', 'betaprime', 'bradford', 'burr', 'burr12', 'cauchy', 'chi',
                 'chi2', 'cosine', 'dgamma', 'dweibull', 'erlang', 'expon', 'exponnorm', 'exponweib', 'exponpow', 'f',
                 'fatiguelife', 'fisk', 'foldcauchy', 'foldnorm', 'frechet_r', 'frechet_l', 'genlogistic', 'genpareto',
                 'gennorm', 'genexpon', 'genextreme', 'gausshyper', 'gamma', 'gengamma', 'genhalflogistic', 'gilbrat',
                 'gompertz', 'gumbel_r', 'gumbel_l', 'halfcauchy', 'halflogistic', 'halfnorm', 'halfgennorm',
                 'hypsecant', 'invgamma', 'invgauss', 'invweibull', 'johnsonsb', 'johnsonsu', 'kstwobign', 'laplace',
                 'levy', 'levy_l', 'logistic', 'loggamma', 'loglaplace', 'lognorm', 'lomax', 'maxwell', 'mielke',
                 'nakagami', 'ncx2', 'ncf', 'nct', 'norm', 'pareto', 'pearson3', 'powerlaw', 'powerlognorm',
                 'powernorm', 'rdist', 'reciprocal', 'rayleigh', 'rice', 'recipinvgauss', 'semicircular', 't', 'triang',
                 'truncexpon', 'truncnorm', 'tukeylambda', 'uniform', 'vonmises', 'vonmises_line', 'wald',
                 'weibull_min', 'weibull_max']

results = []

# fit every distribution to the dataset
for i in list_of_dists:
    dist = getattr(stats, i)
    param = dist.fit(LogData)
    a = stats.kstest(LogData, i, args=param)
    results.append((i, a[0], a[1]))

# sort the results, best fitting is on the top
results.sort(key=lambda x: float(x[2]), reverse=True)
for j in results:
    print("{}: statistic= {}, pvalue= {}".format(j[0], j[1], j[2]))

Wet and dry weather

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import gmean
import statistics as st

# Dictionary of Target Values for each bacteria
targetGM = {"Fecal Coliform": 200, "Fecal coliform": 200, "Fecal\nColiform":200,
          "Enterococci": 35, "Enterococcus":35,
          "E.coli": 126, "E.Coli": 126, "Ecoli": 126, "Escherichia coli":126, "Escherichia\nColi":126,
          "Total Coliform": 200}
targetSTV = {"Fecal Coliform": 770, "Fecal coliform": 770, "Fecal\nColiform":770,
          "Enterococci": 130, "Enterococcus":130,
          "E.coli": 410, "E.Coli": 410, "Ecoli": 410, "Escherichia coli":410, "Escherichia\nColi":410,
          "Total Coliform": 770}

def target_number(iterable, targetvalue): # if the bacteria is greater than the target value, count it.
    if len(iterable) < 1:
        return "NA"
    else:
        count = 0
        for i in iterable:
            if i > targetvalue:
                count = count + 1
        return count

def target_percentile(iterable, targetvalue): # calculate the percent of bacteria that is greater than the target value
    if len(iterable) < 1:
        return "NA"
    else:
        count = 0
        for i in iterable:
            if i >= targetvalue:
                count += 1
        totallist = len(iterable)
        return round((count / totallist) * 100, 1)

# Statistical Threshold Value
def stv(iterable):
    # log based 10 : STV = 10 ** ( avg(log values) + 1.282 * std(log values) )
    print("STV LIST: ",iterable)
    iterable = [i for i in iterable if i != 0]
    print("NEW STV LIST: ", iterable)
    if len(iterable) < 3:
        print("THIS IS HITTING THE <3 MARK")
        return "NA"
    else:
        b = np.log10(iterable)
        eq = 10 ** (st.mean(b) + (1.282 * st.stdev(b)))
        return round(eq,1)

def geomean(iterable): # Geometric mean
    if len(iterable) < 5:
        return "NA"
    else:
        return round(float(gmean(iterable)),1)

def maxvalue(iterable): # max value of the list
    if len(iterable) < 2:
        return 'NA'
    else:
        return round(float(max(iterable)),2)

def minvalue(iterable): # min value of the list
    if len(iterable) < 2:
        return 'NA'
    else:
        return round(float(min(iterable)),2)

# Create a list of lists for the bacteria, location, month, etc
LIST = []


# Uses the month, bacteria, and location, get the geometric mean, STV, average, and percentiles
# and exports this information into a new pandas dataframe
def makingdatasheet(bacteria, locationID):
    # ALL DATA FIRST
    # Matching the correct columns with the parameters from the function
    DATA = (df.loc[(df['Bacteria'] == (bacteria))
                   & (df['Location'] == (locationID))])

    # Make a list of all of the data (MPN/100mL) with the same criteria (bacteria and location)
    DataList = DATA['Data'].tolist()


    print("\nBacteria: " + str(bacteria))
    print("Location ID: " + str(locationID))
    print("Data List for all values ",DataList)

    if not DataList:
        print("No data\n")

    else:
        # Print to check the loop
        print("Length of total data ",len(DataList))

        # Percentiles
        p0 = round(np.percentile(DataList, 0),1)
        p5 = round(np.percentile(DataList, 5),1)
        p25 = round(np.percentile(DataList, 25),1)
        p50 = round(np.percentile(DataList, 50),1)
        p75 = round(np.percentile(DataList, 75),1)
        p90 = round(np.percentile(DataList, 90),1)
        p100 = round(np.percentile(DataList, 100),1)

        # Target Value Caculations
        #print("DATA LIST NOW ",DataList)
        targetnumberGM = target_number(DataList, targetGM[str(bacteria)])
        targetpercentGM = target_percentile(DataList, targetGM[str(bacteria)])
        targetnumberSTV = target_number(DataList, targetSTV[str(bacteria)])
        targetpercentSTV = target_percentile(DataList, targetSTV[str(bacteria)])
        GM = geomean(DataList)
        STV = stv(DataList)
        print('scipy GM ',GM)
        print("Target GM",targetGM[str(bacteria)])
        print("Target STV", targetSTV[str(bacteria)])
        print("Target Number GM ",targetnumberGM)
        print("Target Percent GM ",targetpercentGM)
        print("Target Number STV ",targetnumberSTV)
        print("Target Percent STV ",targetpercentSTV)

        # DRY WEATHER DATA
        drydata = (df.loc[(df['Bacteria'] == (bacteria))
                          & (df['Location'] == (locationID))
                          & (df['Sum of Rain for Two Days'] < .1)])
        DryDataList = drydata['Data'].tolist()
        print("Length of dry data ", len(DryDataList))
        drytargetnumberGM = target_number(DryDataList, targetGM[str(bacteria)])
        drytargetpercentGM = target_percentile(DryDataList, targetGM[str(bacteria)])
        drytargetnumberSTV = target_number(DryDataList, targetSTV[str(bacteria)])
        drytargetpercentSTV = target_percentile(DryDataList, targetSTV[str(bacteria)])
        drySTV = stv(DryDataList)
        dryGM = geomean(DryDataList)
        drymin = minvalue(DryDataList)
        drymax = maxvalue(DryDataList)

        # WET WEATHER DATA
        wetdata = (df.loc[(df['Bacteria'] == (bacteria))
                          & (df['Location'] == (locationID))
                          & (df['Sum of Rain for Two Days'] >=.1)])
        WetDataList = wetdata['Data'].tolist()
        print("Length of wet data ", len(WetDataList))
        wettargetpercentGM = target_percentile(WetDataList, targetGM[str(bacteria)])
        wettargetnumberGM = target_number(WetDataList, targetGM[str(bacteria)])
        wettargetpercentSTV = target_percentile(WetDataList, targetSTV[str(bacteria)])
        wettargetnumberSTV = target_number(WetDataList, targetSTV[str(bacteria)])
        wetSTV = stv(WetDataList)
        wetGM = geomean(WetDataList)
        wetmin = minvalue(WetDataList)
        wetmax = maxvalue(WetDataList)

        return LIST.append([locationID, bacteria, targetGM[str(bacteria)], targetSTV[str(bacteria)],
                            len(DataList),
                            float(p0), float(p5), float(p25), float(p50), float(p75), float(p90), float(p100),
                            targetnumberGM, targetpercentGM,
                            targetnumberSTV, targetpercentSTV,
                            GM, STV,
                            len(DryDataList), len(WetDataList),
                            drytargetnumberGM, wettargetnumberGM,
                            drytargetnumberSTV, wettargetnumberSTV,
                            drytargetpercentGM, wettargetpercentGM,
                            drytargetpercentSTV, wettargetpercentSTV,
                            dryGM, wetGM,
                            drySTV, wetSTV,
                            drymin, drymax, wetmin, wetmax])



#### Create a Pandas Dataframe of the data.
exportingdata = pd.DataFrame(columns=["Location", "Bacteria", "Target Value for GM", "Target Value for STV",
                                      "Number of Observations",
                                      "P0", "P5", "P25", "P50", "P75", "P90", "P100",
                                      "Number Exceeding Target GM", "Percent that exceed the Target GM",
                                      "Number Exceeding Target STV", "Percent that exceed the Target STV",
                                      "Geo Mean", "STV",
                                      "Number of Dry Weather Samples", "Number of Wet Weather Samples",
                                      "Number Dry Samples Exceeding Target GM", "Number Wet Samples Exceeding Target GM",
                                      "Number Dry Samples Exceeding Target STV", "Number Wet Samples Exceeding Target STV",
                                      "Percent of Dry Samples that exceed the Target GM", "Percent of Wet Samples that exceed the Target GM",
                                      "Percent of Dry Samples that exceed the Target STV", "Percent of Wet Samples that exceed the Target STV",
                                      "Dry Samples GM", "Wet Samples GM", "Dry Samples STV", "Wet Samples STV",
                                      "Low of Dry Samples", "High of Dry Samples",
                                      "Low of Wet Samples","High of Wet Samples"])


AllData = pd.read_excel(r'/All Data.xlsx')
#### Create a pandas dataframe
df = pd.DataFrame(AllData)

# Make a list of only the unique locations and bacteria types
LocationList = df['Location'].unique().tolist()
BacteriaList = df['Bacteria'].unique().tolist()

print('Location List:\n', LocationList)
print('BacteriaList:\n', BacteriaList)

for L in range(len(LocationList)): #loops through all the unique location
    for B in range(len(BacteriaList)): #loops through all the unique bacteria
        makingdatasheet(BacteriaList[B], LocationList[L]) # runs it through the function


exportingdata = exportingdata.append(pd.DataFrame(LIST, columns=exportingdata.columns))

##### EXPORT TO EXCEL

### CHANGE THIS TO NAME
file_name = 'AllData_WetandDryData.xlsx'

# saving the excel
exportingdata.to_excel(file_name)
print('\nDataFrame is written to Excel File successfully.')

**Data Visualization**

In [ ]:
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pylab as plt

In [ ]:
exceldata = pd.read_excel('AllDataYearly.xlsx')
df = pd.DataFrame(exceldata)
df

In [ ]:
Ecolidf = df[df['Bacteria']=="E.coli"]
Ecolidf2019 = Ecolidf[Ecolidf['Year']==2019]
EColidf = df[df['Bacteria']=="E.Coli"]
Ecolidf2020 = EColidf[EColidf['Year']==2020]
Ecolidf2021 = EColidf[EColidf['Year']==2021]
#Ecolidf2019.loc[:,['Location','Number of Observations','P0', 'P5', 'P25', 'P50', 'P75', 'P90', 'P100', 'GeoMean', 'STV','Number Exceeding Target GM', 'Percent that Exceed the Target STV']]
#temp = Ecolidf2019.loc[:,['Location','GeoMean']]
temp = Ecolidf2021.loc[:,['Location','Dry Samples GM']]
temp

In [ ]:
ax = plt.gca()
plt.axhline(y=10, color="red", linestyle=":")
plt.xticks(rotation=90)
plt.title("Nearshore | E.coli | 2021 | Dry")
plt.ylabel("cfu/100mL")
ax.set_ylim([-100, 2000])
plt.plot(temp['Location'],temp['Dry Samples GM'],'o', color='black')

#plt.show()